## Setting up conceptual Identity for NTK and BANN 

For this experiment, my aim is to making the case with leveraging NTK with BANN. This case will allow us to place our work in the context of translating GP-NTK to wide deep NN, which is an emerging fields of study recently. After reviewing recent works regarding connections between BNN-GP-wide deep NN, I initially suggested the comparison between He et al. (2020) and our study to show that our BANN model + NTK can provide interpretation via interaction which they could not provide through Bayesian Deep Ensemble via NTK. This experiment could make some interesting point and give some new perspectives. However, before starting the experiment it is important to answer some conceptual questions. Therefore, in this notebook I will try to explain the conceptual identity for NTK and BANN setup that I will use for this experiment. 

### BANN setup

Before starting the experiment, we need to clarify what kind of BANN setup we will going to use. Our BANN is very flexible via computational skeleton blocks that allow us to perform as a classical BNN to Deep Kernel Learning. This flexibility of our structure allows to obtain uncertainty estimates, interaction and make connection as a VI approximation on DGP. But, it is important that before constructing our BNN with blocks we need to remember that we are trying to leverage NTK with BANN in this experiment and wisely decide which setup we need to use. 

NTK is defined through the inner product between the gradients of the network outputs with respect to the network parameters, and by using GP with this deterministic kernel, it captures the behavior of fully-connected deep nets in the infinite width limit trained by gradient descent. Therefore, it is most plausible if we implement NTK to our model when we derived posterior apporximation via our structure with our base set up for BNN structure, namely figure (d) on our paper. In order to make relation with DGP to our model we constructed our model by treating random weights in RF blocks fixed and make weights in FB as a parameter to construct a random feature matrix.

However, with already designed setup for posterior approximation using blocks it is difficult to decide where to implement NTK to our model. If we look at other paper to get a hint where they implemented NTK to their model, He et al. used NTK as a prior kernel to calculate GP posterior for the deep ensemble model and Arora et al. implemeted NTK to convloutional kernel to construct CNTK. Therefore, to implement NTK to our main BNN setup we need to make a modfication to our model without interfering the main context of our suggested block theory. In order to do this, we need to revise our suggested algorithm for this block structure. According to our Algorithm, we simply build our block by replicating RF-FB connection. So to implement NTK in right position we could reconstruct it our algorithm as replication of RF-NTK-FB blocks since FB block takes input of a feature map and NTK makes output of feature map. This seems similar to our Deep Random Feature model with RF-RF-FB blocks, which makes this theory little more to sense.

To summarize, our BANN setup to leverage NTK with BANN would be by constructing our computational skeleton block with replicating RF-NTK-FB. For Other option, we could also construct our computation skeleton block as RF-FB-NTK-FB.

In [ ]:
#Example BANN code, code from Yunyang

n_inputs = n_variables
n_hidden1 = 2
n_hidden2 = 2
n_hidden3 = 2
n_hidden4 = 2
n_hidden5 = 2
n_hidden6 = 2
n_hidden7 = 2
n_hidden8 = 2
n_hidden9 = 2
n_hidden10 = 2
n_hidden11 = 7
n_hidden22 = 8
n_hidden33 = 8
n_hidden44 = 6
n_hidden55 = 9
n_hidden66 = 7
n_hidden77 = 5
n_hidden88 = 10
n_hidden99 = 12
n_hidden1010 = 13
n_outputs = 1
g_scale = 1
thresh = 0.5
dropout_rate = 0.05
n_hiddens = n_hidden1 + n_hidden2 + n_hidden3 + n_hidden4 + n_hidden5 + n_hidden6 + n_hidden7 + n_hidden8 + n_hidden9 + n_hidden10
elems_scale = [0.01] * n_hiddens
g_scalevec = tf.constant(elems_scale)
training = tf.placeholder_with_default(False, shape=(), name='training')
X = tf.placeholder(tf.float32, (None, n_inputs), name='X')
y = tf.placeholder(tf.float32, (None), name = 'y')

This code shows brief understanding how we construct our block structure. It is basically a feed-forward computation structure from the inputs to the outputs as a multi-layer graph with either replication or random weights.

### NTK

This new suggested BNN setup will make us possible to use NTK as a deterministic kernel that captures the behavior of fully-connected deep nets in the infinite width limit. In this setup we could obtain an advantage of getting posterior samples in infinite width limit for implementing NTK to our model. This points out that in this setup NTK would be separate with RF blocks since RF blocks takes the role of random feature approximations for kernels to leverage the expressive power of DGP and FB block takes its role defining randomised priors.

Therefore, by using NTK we could take an advantage of expanding our posterior approximation to infinite width limit and this could help us produce some new perspective on the relation in BNN-GP-wide deep NN.

In [ ]:
#Example NTK code, Code from He et al.
#calculating analytic GP posterior predictives with either NNGP/NTK as prior kernel for an infinite width version of the NN.

analytic_ntkgp_moments, analytic_nngp_moments = predict.gp_inference(
    kernel_fn = kernel_fn,
    x_train = train_data.inputs,
    y_train = train_data.targets,
    x_test = test_data.inputs,
    get = ('ntk', 'nngp'),
    diag_reg = config.NOISE_SCALE**2,
    compute_cov = True
)

predictions = {
    'NTKGP analytic': analytic_ntkgp_moments,
    'NNGP analytic': analytic_nngp_moments
};

Implementing NTK has been very simplified thanks to google's library using JAX. On top of this, He et al. created library called 'bayesian-ntk' that calculates analytic NNGP and NTKGP posterior predictive. Otherwise, we can also define our own ntk with simple code.

### Interaction

Now, with these settings for BNN and NTK, we need to show how our model perform and the significance of our model structure. This part of the experiment will be the last part of this experiment. After constructing the model with our setup for BNN and NTK, we could appply the suggested additive structure with interaction and make BANN-NTK moddel. Then, we can make a comparison on performances and show the advantages of our model to He et al.'s Bayesian Deep Ensemble via NTK. By doing this we could sell our model being **flexible** and **interpretable** at the same time. 

In [ ]:
#Example Interaction code, Code from Yunyang

def get_1interaction(tXtest, nums, simu, pid):
	iXtest = tXtest.copy()
	melem = simu.eval(feed_dict={X:iXtest})
	strength = 0
	x0 = np.linspace(0, 1, nums)
	for i in range(nums):
		iXtest[:, pid] = x0[i]
		selem = simu.eval(feed_dict={X:iXtest})
		strength = strength + np.power(selem - melem, 2)
	stg = np.sqrt(strength * 1.0 / nums)
	return stg

This code shows brief understanding how we calculate friedmann interaction in our additive structure.